In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from konlpy.tag import Okt
import pandas as pd
import re
import logging
from tqdm import tqdm

# 웹소설 제목 변수 설정
webnovel_title = '천산다객-적가천금'         # 웹소설 제목 변수 설정

# 설정 로깅
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

# 불용어 목록 확장
stop_words = set([
    '를', '이', '은', '는', '있다', '하다', '에', 'ㅠ', 'ㅋ',
    '건가', 'ㅎ', '일이', '무슨', '대한', '슈도', '뭔가', '진짜',
    '정말', '생각', '사람', '보고', '누구', '정도', '위해', '때문', '이건',
    '어디', '가장', '아주', '제일', '그냥', '해도', '하나', '얼마나', '자기',
    '부분', '어찌', '저런', '자신', '것', '수', '등', '및', '점'
])

# 단어를 치환하는 것(예를 들어 부수를 부수의로 치환해서 계산하는 것임)
word_mapped = {"천산": "천산다객","산다": "천산다객","다객": "천산다객",
               "리가": "강리","강원": "강원백","은지": "은지여","지여": "은지여",
               "형": "희형","강유": "강유요","여장": "여장성"}

# 데이터 로드
data = pd.read_excel(f'{webnovel_title}-Total_collected_data.xlsx', na_values=['NaN'])
data = data.dropna()  # NaN 값을 가진 행 제거
data.reset_index(drop=True, inplace=True)  # 인덱스 재설정

# 텍스트 열 추출
texts = data["Review"].astype(str).tolist()

# Okt 초기화
okt = Okt()

# 텍스트 전처리 함수
def preprocess(text):
    # 특수 문자와 숫자 제거
    text = re.sub(r"[^가-힣\s]", "", text)
    # Okt를 사용하여 토큰화
    tokens = okt.nouns(text)
    # 단어 치환
    tokens = [word_mapped.get(token, token) for token in tokens]
    # 불용어 제거
    tokens = [token for token in tokens if token not in stop_words and len(token) > 1]
    return tokens

# 전처리된 텍스트
processed_texts = [preprocess(text) for text in tqdm(texts)]

# 사전과 말뭉치 생성
dictionary = corpora.Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]

# LDA 모델 설정
num_topics = 5
chunksize = 4000
passes = 30
iterations = 500
eval_every = 10

# eval_every = None

# LDA 모델 학습
model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    chunksize=chunksize,
    alpha="auto",
    eta="auto",
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

# 토픽 코히런스 계산
top_topics = model.top_topics(corpus)
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print("Average topic coherence: %.4f." % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

# 시각화 준비
lda_visualization = gensimvis.prepare(model, corpus, dictionary, n_jobs=1)
pyLDAvis.save_html(lda_visualization, f"{webnovel_title}-LDA.html")